<a href="https://colab.research.google.com/github/geekymonk123/Real-time-traffic-monitoring-system/blob/main/MainProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U kaleido

In [2]:
  #!pip install easyocr
import cv2
import torch
import numpy as np
import time
import csv
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T
import easyocr
from google.colab.patches import cv2_imshow

# Load the pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define a transformation for the input image
transform = T.Compose([T.ToTensor()])

# Initialize EasyOCR for license plate OCR
reader = easyocr.Reader(['en'], gpu=True)

# Function to perform object detection and classification
def detect_objects(image):
    input_tensor = transform(image).to(device)
    input_batch = input_tensor.unsqueeze(0)

    with torch.no_grad():
        predictions = model(input_batch)

    return predictions

# Function to calculate the absolute speed of a vehicle
def calculate_absolute_speed(prev_position, current_position, time_diff, focal_length):
    distance_pixels = np.linalg.norm(current_position - prev_position)
    distance_meters = (focal_length * 2) / distance_pixels
    speed_mps = distance_meters / time_diff
    speed_kph = speed_mps * 3.6
    return speed_kph

# Function to perform OCR on license plates
def ocr_license_plate(image, coordinates):
    x, y, w, h = int(coordinates[0]), int(coordinates[1]), int(coordinates[2]), int(coordinates[3])
    plate_image = image[y:h, x:w]

    gray_plate = cv2.cvtColor(plate_image, cv2.COLOR_RGB2GRAY)
    result = reader.readtext(gray_plate, detail=0)
    license_plate_number = ""
    confidence_score = 0.0

    for res in result:

        if len(res) > 6:
            license_plate_number = res


    return license_plate_number, confidence_score

# Function to process video frames and write results to CSV
def process_frame(frame, prev_positions, prev_time, writer, video_capture, focal_length):
    predictions = detect_objects(frame)

    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()

    current_time = time.time()
    time_diff = current_time - prev_time

    for box, label in zip(boxes, labels):
        box_center = np.array([(box[0] + box[2]) / 2, (box[1] + box[3]) / 2])
        if label in prev_positions:
            prev_position = prev_positions[label]
            speed = calculate_absolute_speed(prev_position, box_center, time_diff, focal_length)

            license_plate_number, confidence_score = ocr_license_plate(frame, box)

            # Write license plate number and confidence level to CSV
            if license_plate_number:
                writer.writerow({'Frame': int(video_capture.get(cv2.CAP_PROP_POS_FRAMES)),
                                 'Label': label,
                                 'License Plate': license_plate_number,
                                 'Confidence Score': confidence_score,
                                 'X': box_center[0],
                                 'Y': box_center[1],
                                 'Speed (km/h)': speed})

                # Draw bounding box around the detected object
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

                # Display the license plate number and speed on the frame
                text = f'{license_plate_number} {speed:.2f} km/h'
                cv2.putText(frame, text, (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        prev_positions[label] = box_center

    return frame

# Function to process real-time video stream
def process_realtime_video(video_path, focal_length, output_csv):
    video_capture = cv2.VideoCapture(video_path)
    prev_positions = {}
    prev_time = time.time()

    # Get video frame properties
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Open CSV file for writing license plate predictions
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Frame', 'Label', 'License Plate', 'Confidence Score', 'X', 'Y', 'Speed (km/h)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            processed_frame = process_frame(frame, prev_positions, prev_time, writer, video_capture, focal_length)

            cv2_imshow(processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            prev_time = time.time()

    video_capture.release()
    cv2.destroyAllWindows()

# Run the real-time video processing function
video_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/pexels_videos_2103099 (2160p) (1) (online-video-cutter.com).mp4'
focal_length = 1000.0  # Adjust as per your camera setup
output_csv = '/content/drive/MyDrive/Colab Notebooks/Dataset/Untitled spreadsheet - Sheet1.csv'
process_realtime_video(video_path, focal_length, output_csv)


ModuleNotFoundError: No module named 'easyocr'

Object Motion detection

In [ ]:
import torch
import torchvision
import cv2
import numpy as np
from torchvision.transforms import functional as F
from PIL import Image
from google.colab.patches import cv2_imshow
# Define the class labels for Faster R-CNN
faster_rcnn_classes = [
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Load the pre-trained Faster R-CNN model
fasterrcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
fasterrcnn_model.eval()

# Function to perform inference using Faster R-CNN
def detect_objects_fasterrcnn(frame, model):
    image = Image.fromarray(frame)
    image_tensor = F.to_tensor(image)

    with torch.no_grad():
        outputs = model([image_tensor])

    return outputs[0]

# Function to draw bounding boxes and labels
def draw_boxes(frame, detections, classes):
    for box, label, score in zip(detections['boxes'], detections['labels'], detections['scores']):
        if score > 0.5:
            x1, y1, x2, y2 = map(int, box.tolist())
            label = f'{classes[label.item()]}: {score:.2f}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    return frame

# Open the video file
video_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Video.mp4'
cap = cv2.VideoCapture(video_path)

# Prepare for saving output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference using Faster R-CNN
    detections_fasterrcnn = detect_objects_fasterrcnn(frame, fasterrcnn_model)

    # Draw bounding boxes and labels
    frame_fasterrcnn = draw_boxes(frame.copy(), detections_fasterrcnn, faster_rcnn_classes)

    # Write the frame with detections to the output video
    out.write(frame_fasterrcnn)

    # Display the frame
    cv2_imshow(frame_fasterrcnn)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
!pip install pytesseract
!apt-get install tesseract-ocr


In [ ]:
import pytesseract
!pip install easyocr
import easyocr
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
import re
from difflib import SequenceMatcher

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to perform OCR using Tesseract
def ocr_tesseract(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray).strip()
    return text

# Function to perform OCR using EasyOCR
def ocr_easyocr(image):
    result = reader.readtext(image)
    text = ' '.join([res[1] for res in result])
    return text

# Function to preprocess text
def preprocess_text(text):
    # Remove extra whitespace, newlines, and non-alphanumeric characters
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.lower()

# Function to calculate similarity ratio
def calculate_similarity(gt_text, pred_text):
    return SequenceMatcher(None, gt_text, pred_text).ratio()

# Function to calculate accuracy
def calculate_accuracy(gt_texts, pred_texts):
    correct = sum([1 for gt, pred in zip(gt_texts, pred_texts) if gt == pred])
    return correct / len(gt_texts)

# Sample images and ground truth texts
image_paths = [
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB1.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB11.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB12.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB13.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB14.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB15.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB16.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB17.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB18.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB19.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB20.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB21.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB22.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB23.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB25.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB26.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB27.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB28.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB2.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB3.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB4.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB5.jpg',
      '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB6.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB8.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/WB9.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN1.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN2.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN3.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN4.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN5.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN6.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN7.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/DN8.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ2.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ4.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ5.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ6.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ7.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ9.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/RJ10.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN1.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN2.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN3.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN4.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN5.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN6.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN7.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN8.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN9.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/TN10.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP1.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP2.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP3.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP4.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP5.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP8.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP10.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP11.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP12.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP14.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP15.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP16.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP17.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP19.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP20.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP21.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP22.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP23.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP24.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP25.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP26.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP27.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP28.jpg',
     '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/UP29.jpg'

]
gt_texts = [
    'WB26T6688','WB08A4101','WB26S5093',
    'WB26K2760','WB36G6682',
    'WB12AX2987','WB06J9177','WB02AJ4534',
    'WB06J2192','WB061061','WB171717',
    'WB02W9S3S','WB20H0271','WB20AG9041',
    'WB06H2237','WB02A03017','WB02AD4162',
    'WB02AM8548','WB02M8447','WB70G3763','WB02X8795',
    'WB06G4120','WB02AH4655','WB06J4432','WB02AA5580',
    'DN09H2191','DN09J0548','DN09H0968','DN09J0032','DN09D2724',
    'DN0901438','DN0909704','DN09C0868',
    'RJ14CW4115','RJ20CD5118','RJ14CF5807','RJ23CB2272',
    'RJ23CB4320','RJ20UB0137','RJ14CM3730',
    'TN63BV7954','TN76V1978','TN02AU9295','TN10AR0131',
    'TN47T4464','TN19F0816','TN07BT5778','TN07AP0659','TN19Q0835','TN69AJ4455',
    'UP57H8173','UP85BN4203','UP80FA1666','UP16U3849','UP32EC5577','UP16AJ0704','UP78DB3730',
    'UP53AK1161','UP14CV7920','UP16BD4866','UP79L0567','UP32KW7325','UP32CX3400','UP45Q3028',
    'UP14AK8604','UP52AH9705','UP11AY0646','UP78ED0449','UP27Z7322','UP32BF3577','UP14AX9754',
    'UP70ET7816','UP32HY6077','UP14AF3804'
    # Add corresponding ground truth texts
]

tesseract_texts = []
easyocr_texts = []
tesseract_accuracies = []
easyocr_accuracies = []
for image_path,gt_text in zip(image_paths,gt_texts):
    image = cv2.imread(image_path)

    # Perform OCR using Tesseract
    tesseract_text = ocr_tesseract(image)
    tesseract_texts.append(tesseract_text)

    # Perform OCR using EasyOCR
    easyocr_text = ocr_easyocr(image)
    easyocr_texts.append(easyocr_text)
     # Preprocess texts
    gt_text_clean = preprocess_text(gt_text)
    tesseract_text_clean = preprocess_text(tesseract_text)
    easyocr_text_clean = preprocess_text(easyocr_text)

    # Calculate similarity ratios
    tesseract_similarity = calculate_similarity(gt_text_clean, tesseract_text_clean)
    easyocr_similarity = calculate_similarity(gt_text_clean, easyocr_text_clean)

    # Append accuracies (similarity ratios)
    tesseract_accuracies.append(tesseract_similarity)
    easyocr_accuracies.append(easyocr_similarity)

# Calculate average accuracies
average_tesseract_accuracy = np.mean(tesseract_accuracies)
average_easyocr_accuracy = np.mean(easyocr_accuracies)

print("Tesseract OCR Accuracy:", average_tesseract_accuracy * 100)
print("EasyOCR Accuracy:", average_easyocr_accuracy * 100)


In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define COCO class names (first class is the background)
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',
    'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
    'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet',
    'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# Define the image transformation
transform = T.Compose([
    T.ToTensor()
])

# Load the image
image_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/Classification.jpg'
image = Image.open(image_path)
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    predictions = model(image_tensor)

# Get the predictions
pred_boxes = predictions[0]['boxes'].cpu().numpy()
pred_scores = predictions[0]['scores'].cpu().numpy()
pred_classes = predictions[0]['labels'].cpu().numpy()

# Filter out low confidence scores
threshold = 0.5
filtered_boxes = pred_boxes[pred_scores >= threshold]
filtered_classes = pred_classes[pred_scores >= threshold]

# Draw bounding boxes on the original image
output_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

for i, box in enumerate(filtered_boxes):
    if COCO_INSTANCE_CATEGORY_NAMES[filtered_classes[i]] == 'car':
        x1, y1, x2, y2 = box
        cv2.rectangle(output_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f'{COCO_INSTANCE_CATEGORY_NAMES[filtered_classes[i]]}: {pred_scores[i]:.2f}'
        cv2.putText(output_image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Display the image
plt.figure(figsize=(12, 8))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# Optionally save the output image
output_image_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/output_car_image.jpg'
cv2.imwrite(output_image_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))


In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define COCO class names (first class is the background)
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',
    'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
    'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet',
    'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# Define the image transformation
transform = T.Compose([
    T.ToTensor()
])

# Load the image
image_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Images/Classification.jpg'
image = Image.open(image_path)
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    predictions = model(image_tensor)

# Get the predictions
pred_boxes = predictions[0]['boxes'].cpu().numpy()
pred_scores = predictions[0]['scores'].cpu().numpy()
pred_classes = predictions[0]['labels'].cpu().numpy()

# Filter out low confidence scores
threshold = 0.94
filtered_boxes = pred_boxes[(pred_scores >= threshold) & (pred_classes == COCO_INSTANCE_CATEGORY_NAMES.index('car'))]
filtered_scores = pred_scores[(pred_scores >= threshold) & (pred_classes == COCO_INSTANCE_CATEGORY_NAMES.index('car'))]
filtered_classes = pred_classes[(pred_scores >= threshold) & (pred_classes == COCO_INSTANCE_CATEGORY_NAMES.index('car'))]

# Draw bounding boxes on the original image
output_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

for i, box in enumerate(filtered_boxes):
    x1, y1, x2, y2 = box
    cv2.rectangle(output_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    label = f'{COCO_INSTANCE_CATEGORY_NAMES[filtered_classes[i]]}: {filtered_scores[i]:.2f}'
    cv2.putText(output_image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Display the image
plt.figure(figsize=(12, 8))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# Optionally save the output image
output_image_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/output_car_image.jpg'
cv2.imwrite(output_image_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image with bounding boxes
image_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/download (37).png'
image = cv2.imread(image_path)

# Create a mask where the green lines are located
lower_green = np.array([0, 255, 0], dtype=np.uint8)
upper_green = np.array([0, 255, 0], dtype=np.uint8)
mask = cv2.inRange(image, lower_green, upper_green)

# Dilate the mask to cover the text and bounding box
kernel = np.ones((3, 3), np.uint8)
mask = cv2.dilate(mask, kernel, iterations=1)

# Inpaint the image using the mask
inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)

# Convert the image from BGR to RGB
output_image = cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(12, 8))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# Optionally save the output image
output_image_path = '/mnt/data/output_image_without_boxes.png'
cv2.imwrite(output_image_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
